# scaler 사용법 정리 : 예측할 때 하나니까 미리 맞춰놓아야 한다.
하나밖에 없으니까 minmax가 안 된다
- fit_transform 두 가지 역할 동시에 한다.
- fit : minmax 할 data 들고와
- transform : 변환해
나는 sclaer 사용했을 때 이상할 수도 있겠는데?????????????????test, 안에서만 하고 train 안에서만 했으니까.

# one-hot indexing 하는 방법 정리
- pandas.get_dummies()
- tensorflow.one_hot()
- 데이터 개수 주의하기!!!!

# cast할 때 계속 졸아서 모른다 정리

# 항상, 데이터 볼 때 결측치(isnull), 이상치(boxplot) 확인

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\ten

In [2]:
bmi_df = pd.read_csv("./data/bmi.csv", skiprows =3)
display(bmi_df) 
# error_bad_lines=False? skiprows?

bmi_df.columns

,label,height,weight
0,1,188,71
1,2,161,68
2,0,178,52
3,2,136,63
4,1,145,52
...,...,...,...
19995,0,163,48
19996,2,139,70
19997,1,150,48
19998,1,189,69


Index(['label', 'height', 'weight'], dtype='object')

In [3]:
bmi_df["thin"] = 0
bmi_df["normal"] = 0
bmi_df["fat"] = 0

bmi_df.loc[bmi_df["label"] == 0, "thin"] = 1
bmi_df.loc[bmi_df["label"] == 1, "normal"] = 1
bmi_df.loc[bmi_df["label"] == 2, "fat"] = 1

In [4]:
display(bmi_df)
train_num = int(bmi_df.shape[0] * 0.8)
print(train_num)

,label,height,weight,thin,normal,fat
0,1,188,71,0,1,0
1,2,161,68,0,0,1
2,0,178,52,1,0,0
3,2,136,63,0,0,1
4,1,145,52,0,1,0
...,...,...,...,...,...,...
19995,0,163,48,1,0,0
19996,2,139,70,0,0,1
19997,1,150,48,0,1,0
19998,1,189,69,0,1,0


16000


In [5]:
# 데이터 준비 + 학습용/테스트용 나누기
# scaler 맞춰놓기
scaler = MinMaxScaler()
scaler.fit(bmi_df[["height", "weight"]])

# x data 
x_data = bmi_df[["height", "weight"]].values
x_data = scaler.fit_transform(x_data)

train_x_data = x_data[:train_num]
test_x_data = x_data[train_num:]

# y data
train_y_data = bmi_df[["thin", "normal", "fat"]][:train_num].values
test_y_data = bmi_df[["thin", "normal", "fat"]][train_num:].values


In [6]:
# 1. 학습

# 1-1. 데이터 셋 준비
train_x_data
train_y_data

# 1-2. placeholder
X = tf.placeholder(shape = [None, 2], dtype = tf.float32)
Y = tf.placeholder(shape = [None, 3], dtype = tf.float32)

# 1-3. weight, bias
W = tf.Variable(tf.random_normal([2,3]), name = "weight")
b = tf.Variable(tf.random_normal([3]), name = "bias")

# 1-4. hypothesis
logit = tf.matmul(X,W)+b
H = tf.nn.softmax(logit)

# 1-5. cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logit,
                                                                labels = Y))

# 1-6. train
train = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

# 1-7. 준비: session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 1-8. 학습
for step in range(30000):
    _, cost_val = sess.run([train, cost], feed_dict = { X : train_x_data,
                                                        Y : train_y_data})
    if step % 3000 == 0:
        print("cost 값은 : {}".format(cost_val))

cost 값은 : 1.3665722608566284
cost 값은 : 0.35388636589050293
cost 값은 : 0.28072211146354675
cost 값은 : 0.24364839494228363
cost 값은 : 0.21998779475688934
cost 값은 : 0.20312057435512543
cost 값은 : 0.19027547538280487
cost 값은 : 0.18005216121673584
cost 값은 : 0.1716541200876236
cost 값은 : 0.16458822786808014


In [7]:
# 2. 정확도 측정 : accuracy

predict = tf.argmax(H, axis = 1) 
correct = tf.equal(predict, tf.argmax(Y, axis = 1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype = tf.float32))
print("정확도 : {}".format(sess.run(accuracy,
                                feed_dict = {X: test_x_data,
                                            Y: test_y_data})))

정확도 : 0.9804999828338623


In [8]:
# 3. 예측
    
predict_x_data = [[170, 60]]

predict_x_data = scaler.transform(predict_x_data)
print(predict_x_data)

result = sess.run(tf.argmax(H, axis = 1), feed_dict = { X: predict_x_data })[0]
print(result)

[[0.625      0.55555556]]
1


# 이 밑에는 왜 안될까
scaler 때문이었다!

In [9]:
# 3. 예측

# 입력값 받아서 예측

T = int(input())

for _ in range(T):
    
    a, b = map(int, input().split())
    
    predict_x_data = [[a, b]]
    predict_x_data = scaler.transform(predict_x_data)
    result = sess.run(tf.argmax(H, axis = 1), feed_dict = { X : predict_x_data })
    
    if result[0] == 0:
        print("thin")
    elif result[0] == 1:
        print("normal")
    else:
        print("fat")

3
180 50
thin
180 90
fat
180 80
normal


In [10]:
a, b = map(int, input().split())

predict_x_data = [[a, b]]
print(predict_x_data)
print(type(predict_x_data))
predict_x_data = scaler.transform(predict_x_data)
print(predict_x_data)
print(type(predict_x_data))
# # predict_x_data = scaler.transform(predict_x_data)
# print(predict_x_data)
# # result = sess.run(tf.argmax(H, axis = 1), feed_dict = { X : predict_x_data })
# # print(result)
# sess.run(tf.argmax(H, axis=1), feed_dict={X : predict_x_data})

30 60
[[30, 60]]
<class 'list'>
[[-1.125       0.55555556]]
<class 'numpy.ndarray'>
